In [1]:
import pandas as pd
from glob import glob
import json
import os

In [2]:
def get_filelst(SAVE_PATH):
    return list(set([i.replace("_0_shot.json","") for i in os.listdir(SAVE_PATH)]))

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(SAVE_PATH, model_name=''):
    
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{SAVE_PATH}/{model_name}_{i}_shot.json'
        try:

            data.append(
                {   
                    # 'kobest_boolq (macro_f1)': 0.35461338879835325
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[model_name]).T
    return df

In [5]:
SAVE_PATH = '../data/result'
filelst = get_filelst(SAVE_PATH)
preferred_order = {'(ours)_llama2_0.1_ALL_ver2': 1, 
                   '(ours)_Synatra-Mixtral-8x7B-v0.1': 2, 
                   '(ours)_Synatra-Mixtral-8x7B-v0.2': 3, 
                   '(beomi)_llama_2_ko_7b': 4,
                   '(beomi)_open-llama-2-ko-7b': 5,
                   '(upstage)_SOLAR-10.7B-v1.0': 6,
                   '(beomi)_OPEN-SOLAR-KO-10.7B':7,
                   '(mistralai)_Mistral-7B-v0.1':8,
                   '(mistralai)_Mixtral-8x7B-v0.1': 9,
                   '(yanolja)_KoSOLAR-10.7B-v0.2': 10,
                   '(yanolja)_EEVE-Korean-10.8B-v1.0':11,
                   '(yanolja)_EEVE-Korean-Instruct-10.8B-v1.0':12}
ordered_list = sorted(filelst, key=lambda x: preferred_order[x])
temp_df = []
for file in ordered_list:
    df = get_df_klue(SAVE_PATH,model_name=file)
    temp_df.append(df)
all_df = pd.concat(temp_df,axis=1).T.reset_index()
all_df.rename(columns={"index":"name"},inplace=True)
all_df.to_csv("data.csv",index=False)

(ours)_llama2_0.1_ALL_ver2
(ours)_Synatra-Mixtral-8x7B-v0.1
(ours)_Synatra-Mixtral-8x7B-v0.2
(beomi)_llama_2_ko_7b
(beomi)_open-llama-2-ko-7b
(upstage)_SOLAR-10.7B-v1.0
(beomi)_OPEN-SOLAR-KO-10.7B
(mistralai)_Mistral-7B-v0.1
(mistralai)_Mixtral-8x7B-v0.1
(yanolja)_KoSOLAR-10.7B-v0.2
(yanolja)_EEVE-Korean-10.8B-v1.0
(yanolja)_EEVE-Korean-Instruct-10.8B-v1.0


In [4]:
import pandas as pd
import pygwalker as pyg
df = pd.read_csv('./data.csv')
walker = pyg.walk(df,hideDataSourceConfig=True, vegaTheme='vega')

Box(children=(HTML(value='<div id="ifr-pyg-00061413260cb480hug1SX6yrTZwecv4" style="height: auto">\n    <head>…